<a href="https://colab.research.google.com/github/mtnman38/tensorflow-examples/blob/master/example_2_ols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 2: Using TensorFlow to perform OLS regression

---

Dustin Landers
---
11/8/2019

First, we will make sure we have TensorFlow 2.0.0 installed. If not, go ahead an pip install that.

In [0]:
#!pip install tensorflow==2.0.0

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


## Building out the model first in statsmodels

Next, we will download the famous good ole mtcars dataset and fit a model using statsmodels. This way we can see what the outcome should look like using a traditional solver for this problem. Then we will see if we can learn a bit about how to use TensorFlow by recreating the problem using the new execution methods of TensorFlow 2.0.

In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [4]:
mtcars = sm.datasets.get_rdataset('mtcars').data

reg_model = smf.ols(formula='mpg ~ disp + hp', data=mtcars)
fitted_model = reg_model.fit()
fitted_model.summary()

/usr/local/lib/python3.6/dist-packages/statsmodels/datasets/utils.py:192: FutureWarning: `item` has been deprecated and will be removed in a future version
  return dataset_meta["Title"].item()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     43.09
Date:                Fri, 08 Nov 2019   Prob (F-statistic):           2.06e-09
Time:                        16:57:11   Log-Likelihood:                -80.309
No. Observations:                  32   AIC:                             166.6
Df Residuals:                      29   BIC:                             171.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     30.7359      1.332     23.083      0.000      28.013      33.459
disp          -0.0303      0.007     -4.098      0.000      -0.045      -0.015
hp            -0.0248      0.013     -1.856      0.074      -0.052       0.003
==============================================================================
Omnibus:                        3.082   Durbin-Watson:                   1.370
Prob(Omnibus):                  0.214   Jarque-Bera (JB):                2.788
Skew:                           0.680   Prob(JB):                        0.248
Kurtosis:                       2.508   Cond. No.                         733.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Recreating the model in TensorFlow

This bit is just for getting our data in to a good format for TensorFlow.

In [0]:
X = patsy.dmatrix('1 + disp + hp', data=mtcars)

This is just reshaping the y vector and assigning them to be TensorFlow constants.

In [0]:
X = tf.constant(X)
y = mtcars["mpg"].values
y = tf.constant(y.reshape((X.shape[0], 1)))

We will make an object from tf.Module to hold our trainable variables. As you can see, we initialize with just a column vector of ones.

In [0]:
class MyRegression(tf.Module):
  
    def __init__(self, initialized_vector = [[1.0] for x in range(X.shape[1])]):
        self.weights = tf.Variable(initialized_vector, dtype=tf.float64)
        
myreg = MyRegression()

We will create two functions for prediction and our loss function. Since this is ordinary least squares, our loss function is just the mean squared error (MSE). Using the tf.function decorator gives us a performance benefit during computation.

In [0]:
@tf.function
def predict(X, weights):
    return tf.matmul(X, weights)

In [0]:
@tf.function
def mse(actual, predictions):
    return tf.reduce_mean(tf.square(actual - predictions))

In [0]:
adam_optimizer = tf.optimizers.Adam(learning_rate=.05)

This is the training loop. Let's see if our coefficients are similar to what we got with the statsmodels.

In [11]:
for epoch in range(5000):

    with tf.GradientTape() as gradient_tape:
        predictions = predict(X, myreg.trainable_variables)
        loss = mse(y, predictions)

    gradient = gradient_tape.gradient(loss, myreg.trainable_variables)
    adam_optimizer.apply_gradients(zip(gradient,
                                       myreg.trainable_variables))
    
print(myreg.trainable_variables)

(<tf.Variable 'Variable:0' shape=(3, 1) dtype=float64, numpy=
array([[ 3.06296210e+01],
       [-3.02806395e-02],
       [-2.43431387e-02]])>,)
